# Credits
__This notebook has been adapted from__
- https://git.skewed.de/count0/graph-tool/-/wikis/installation-instructions#googles-colaboratory

# Installing graph-tool (https://graph-tool.skewed.de)

---


To import a library that's not in Colaboratory by default, you can use `!apt-get install`.

But since graph-tool is not in the official repository, we need to add it to the list.

In [1]:
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

zsh:1: no such file or directory: /etc/apt/sources.list
zsh:1: command not found: apt-key
zsh:1: command not found: apt-get
zsh:1: command not found: apt-get


In [2]:
#python3-cairo from Ubuntu's reposity is linked with a different python version; we need to improvise
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

  Using cached pycairo-1.21.0-cp39-cp39-macosx_10_9_x86_64.whl
  Attempting uninstall: pycairo
    Found existing installation: pycairo 1.21.0
    Uninstalling pycairo-1.21.0:
      Successfully uninstalled pycairo-1.21.0
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [3]:
# import graph_tool.all as gt

# model_name="blumenau_drug"
# g = gt.collection.ns[model_name]
# g.get_edges()


ModuleNotFoundError: No module named 'graph_tool'

# Repository of weighted graphs: https://networks.skewed.de/?tags=Weighted

In [ ]:
import numpy as np
import pandas as pd
import graph_tool.all as gt
from google.colab import files

from google.colab import drive

import os.path
from os import path

def export_all_SISOnweightedGraphs_erode(model_names,download_file,store_on_gdrive,path_g_drive="weightedGraphs/",prefix_to_weight=dict()):
   #Exports in ERODE CSV format all weighted graphs in the list model_names. 
   #The CSV file is either stored on google drive, or downloaded, as required by the parameters (Google drive has precedence on downloading)
   #If the google drive option is chosen, the files will be stored in folder weightedGraphs, or different paths provided in parameter path_g_drive
   #  This should be a relative path to a folder (ending with a slash). E.g., if you pass "weightedGraphs/", we will store files in it in MyDrive/weightedGraphs/


  print('Processing and exporting models')
  print("",model_names)

  #Prepare gdrive
  local_file_path_to_import="erodeOnNet/"
  gdrive_path="/content/drive/MyDrive/"+path_g_drive
  if store_on_gdrive:
    drive.mount('/content/drive')
    if not path.exists(gdrive_path):
      print('creating',gdrive_path)
      os.mkdir(gdrive_path)
    if not path.exists(gdrive_path+"/"+local_file_path_to_import):
      print('creating',gdrive_path+"/"+local_file_path_to_import)
      os.mkdir(gdrive_path+"/"+local_file_path_to_import)

  #Export required weighted graphs
  for model_name in model_names:
    weight_name='weight'
    for key in prefix_to_weight:
      if model_name.startswith(key):
        weight_name=prefix_to_weight[key]
        break
    export_SISOnweightedGraphs_erode(model_name,download_file,store_on_gdrive,gdrive_path,weight_name,local_file_path_to_import)

  #Close gdrive
  if store_on_gdrive:
    drive.flush_and_unmount()

  print('All computations completed')

def export_SISOnweightedGraphs_erode(model_name,download_file,store_on_gdrive,gdrive_path,weight_name,local_file_path_to_import):
  #load model
  print("PROCESSING:",model_name,"...")
  
  g = gt.collection.ns[model_name]
    
    
  add_rev= not g.is_directed()
    
  max_nodes=50
  n_nodes=g.num_vertices()
  if n_nodes > max_nodes:
    print(" MODEL DROPPED BECAUSE IT HAS MORE THAN",max_nodes,"NODES")
    return   
  if not(weight_name in g.edge_properties):
    print(" MODEL DROPPED BECAUSE NOT WEIGHTED")
    return

  model_name = model_name.replace("/","_")
  model_name = model_name.replace(" ","_")
  model_name = model_name.replace("-","_")
  model_name = model_name.replace("(","_")
  model_name = model_name.replace(")","_")

  #store unweighted edges in a dataframe, and add weights
  df = pd.DataFrame(data=g.get_edges(), columns=["source", "target"])
  #df["source"]=df["source"]+1
  #df["target"]=df["target"]+1
  
  if weight_name in g.edge_properties:
    weights=g.edge_properties[weight_name].get_array()
  else:
    #We actually never enter here because we return earlier if the graph is unweighted
    weights=np.ones(len(g.get_edges()))
    print("BEWARE:",model_name,"DOES NOT HAVE WEIGHTS.\n WE SET ALL WEIGHTS TO ONE!")
    model_name=model_name+"_defaultWeights"
  
  df['weight']=weights

  df0 = pd.DataFrame([[n_nodes,n_nodes,n_nodes]], columns=["source", "target",'weight'])
  df=pd.concat([df0, df])
  df.reset_index(drop=True, inplace=True)
  #print(df)
    
  #compute file names
  erode_file_name_local=local_file_path_to_import+model_name+'._ode'
  erode_file_name=erode_file_name_local
  import_erode_file_name="import_"+model_name+'.ode'
    
  if store_on_gdrive:
    erode_file_name=gdrive_path+erode_file_name
    import_erode_file_name=gdrive_path+import_erode_file_name
    
  print("  ","Graph processed, and data structure created")

  #create csv file and download if required
  #df.to_csv( csv_file_name ,index=False,header=False)
  #dfB.to_csv(csv_file_nameB,index=False,header=False)

    
    
  # Create ERODE file to import the large ERODE one
  print(import_erode_file_name)
  with open(import_erode_file_name, 'w') as fp:
    fp.write('begin model import_'+model_name+"\n")
    fp.write(' load(fileIn="'+erode_file_name_local+'")\n')
    fp.write(' write(fileOut="ODE/'+model_name+'._ode",format=ODE)\n')
    fp.write(' reduceFE(reducedFile="FE_prep/'+model_name+'_FE._ode", prePartition=USER,csvFile="FE_prep.csv")\n')
    fp.write(' reduceFE(reducedFile="FE/'+model_name+'_FE._ode", prePartition=USER,csvFile="FE.csv")\n')    
    fp.write('end model\n')
    
  #Rates
  p_b="1" #infection
  #p_a="2" #vaxination
  p_g="0.01" #recovery  
  #p_eta="0.02" #Susceptibilization  

  # Create a large ERODE file containing the SIR_VAX on network  
  print(erode_file_name)
  with open(erode_file_name, 'w') as fp:
    fp.write('begin model '+model_name+"\n")
    
    fp.write(' begin parameters\n')
    fp.write('//  b = '+p_b+'//Infection is given implicitly by the weights\n')
    fp.write('  g = '+p_g+'//Recovery\n')    
    # for (edge,weight) in zip(g.get_edges(),weights):
    #     s=str(edge[0])
    #     t=str(edge[1])
    #     w=str(weight)
    #     fp.write('  i_'+s+'_to_'+t+' = '+w+'\n')
    #     if add_rev:
    #         fp.write('  i_'+t+'_to_'+s+' = '+w+'\n')
    fp.write(' end parameters\n')
    
    fp.write(' begin init\n')
    for n in range(n_nodes):
        fp.write('  S'+str(n)+'\n')
        fp.write('  I'+str(n)+'\n')
        #fp.write('  R'+str(n)+'\n')
    fp.write(' end init\n')
    
    fp.write(' begin partition\n')
    fp.write('  {S0},\n')
    fp.write('  {I0}\n')
    #fp.write('  {R0}\n')
    # fp.write('  {')
    # for n in range(n_nodes-1):
    #   fp.write('S'+str(n)+',')
    # fp.write('S'+str(n_nodes-1))
    # fp.write('}\n')
    # fp.write('  {')
    # for n in range(n_nodes-1):
    #   fp.write('I'+str(n)+',')
    # fp.write('S'+str(n_nodes-1))
    # fp.write('}\n')
    fp.write(' end partition\n')
    
    
    fp.write(' begin reactions\n')
    # fp.write('\n  //Vaxinations\n')
    # for n in range(n_nodes):
    #     n_str=str(n)
    #     #S1 -> R1 , a [vax1]
    #     fp.write('  S'+n_str+' -> I'+n_str+' , a [vax_'+n_str+']\n')
    fp.write('\n  //Recovery\n')
    for n in range(n_nodes):
        n_str=str(n)
        #I1 -> R1 , 0.01 [recovery1]
        #fp.write('  I'+n_str+' -> R'+n_str+' , '+p_g+' [recovery_'+n_str+']\n')
        fp.write('  I'+n_str+' -> S'+n_str+' , '+p_g+' [recovery_'+n_str+']\n')

    # fp.write('\n  //Susceptibilization \n')
    # for n in range(n_nodes):
    #     n_str=str(n)
    #     #R1 -> S1 , 0.02 [susc1]
    #     fp.write('  R'+n_str+' -> S'+n_str+' , '+p_eta+' [susc_'+n_str+']\n')
    

    fp.write('\n  //Cross-site infections \n')
    for edge,weight in zip(g.get_edges(),weights):
        s=str(edge[0])
        t=str(edge[1])
        w=str(weight)
        fp.write(    '  S'+t+' + I'+s+' -> I'+t+' + I'+s+' , '+w+' [infection_'+s+'_to_'+t+']\n')
        if add_rev:
            fp.write('  S'+s+' + I'+t+' -> I'+s+' + I'+t+' , '+w+'  [infection_'+t+'_to_'+s+']\n')
    fp.write(' end reactions\n')    
    fp.write('end model\n')    
    
  
  if download_file and not store_on_gdrive:
    files.download(erode_file_name)
    files.download(import_erode_file_name)

  print("  ","All files exported")

    
  
#Choose the models to consider
# Just https://networks.skewed.de/net/blumenau_drug
model_names=["blumenau_drug"]
#model_names=["rhesus_monkey"]
#model_names=["ambassador/KINSHIP"]
#model_names=["dom/Archie_2006h"]
#model_names=["libimseti"]
#model_names=["twitter_higgs/mention"]
#model_names=["cattle"]
#model_names=["blumenau_drug","cattle","train_terrorists"]

#All weighted networks on 21/07/2022 from smallest number of nodes to largest#
# Taken from https://networks.skewed.de/?tags=Weighted after clicking on 'Nodes'
# model_names=[
#              "packet_delays","ambassador/1985_1989","ambassador/1990_1994","ambassador/1995_1999","ambassador/2000","ambassador/2001","ambassador/2002","ambassador/2003","ambassador/2004","ambassador/2005","ambassador/2006","ambassador/2007","ambassador/KINSHIP","ambassador/OPERATION_ID","ambassador/TIE_EXTINGUISH","ambassador/TIE_YEAR",
#              "rhesus_monkey","kangaroo",
#              "sp_baboons/observational","sp_baboons/sensor","bison","cattle","moreno_sheep","college_freshmen","hens",
#              "freshmen/t0","freshmen/t2","freshmen/t3","freshmen/t5","freshmen/t6",
#              "windsurfers","macaques","train_terrorists","highschool","blumenau_drug","lesmis","baseball/player-player","baseball/user-provider","sp_office","game_thrones",
#              "moviegalaxies/1","moviegalaxies/2","moviegalaxies/3","moviegalaxies/5","moviegalaxies/6","moviegalaxies/7","moviegalaxies/8","moviegalaxies/9","moviegalaxies/10","moviegalaxies/11","moviegalaxies/13","moviegalaxies/14","moviegalaxies/15","moviegalaxies/17","moviegalaxies/19","moviegalaxies/21","moviegalaxies/22","moviegalaxies/23","moviegalaxies/24","moviegalaxies/25","moviegalaxies/26","moviegalaxies/27","moviegalaxies/28","moviegalaxies/30","moviegalaxies/31","moviegalaxies/32","moviegalaxies/33","moviegalaxies/34","moviegalaxies/36","moviegalaxies/38","moviegalaxies/39","moviegalaxies/40","moviegalaxies/41","moviegalaxies/42","moviegalaxies/43","moviegalaxies/44","moviegalaxies/45","moviegalaxies/47","moviegalaxies/48","moviegalaxies/49","moviegalaxies/50","moviegalaxies/51","moviegalaxies/52","moviegalaxies/54","moviegalaxies/57","moviegalaxies/58","moviegalaxies/59","moviegalaxies/60","moviegalaxies/61","moviegalaxies/62","moviegalaxies/63","moviegalaxies/64","moviegalaxies/65","moviegalaxies/66","moviegalaxies/67","moviegalaxies/68","moviegalaxies/69","moviegalaxies/70","moviegalaxies/71","moviegalaxies/72","moviegalaxies/73","moviegalaxies/74","moviegalaxies/76","moviegalaxies/80","moviegalaxies/81","moviegalaxies/82","moviegalaxies/83","moviegalaxies/84","moviegalaxies/85","moviegalaxies/86","moviegalaxies/87","moviegalaxies/88","moviegalaxies/89","moviegalaxies/91","moviegalaxies/92","moviegalaxies/93","moviegalaxies/97","moviegalaxies/98","moviegalaxies/99","moviegalaxies/100","moviegalaxies/101","moviegalaxies/102","moviegalaxies/103","moviegalaxies/104","moviegalaxies/106","moviegalaxies/107","moviegalaxies/108","moviegalaxies/109","moviegalaxies/110","moviegalaxies/111","moviegalaxies/112","moviegalaxies/113","moviegalaxies/114","moviegalaxies/115","moviegalaxies/116","moviegalaxies/117","moviegalaxies/118","moviegalaxies/119","moviegalaxies/120","moviegalaxies/122","moviegalaxies/123","moviegalaxies/124","moviegalaxies/125","moviegalaxies/126","moviegalaxies/127","moviegalaxies/128","moviegalaxies/129","moviegalaxies/130","moviegalaxies/131","moviegalaxies/132","moviegalaxies/133","moviegalaxies/134","moviegalaxies/135","moviegalaxies/136","moviegalaxies/137","moviegalaxies/138","moviegalaxies/139","moviegalaxies/141","moviegalaxies/142","moviegalaxies/143","moviegalaxies/144","moviegalaxies/145","moviegalaxies/146","moviegalaxies/147","moviegalaxies/148","moviegalaxies/149","moviegalaxies/150","moviegalaxies/151","moviegalaxies/152","moviegalaxies/153","moviegalaxies/154","moviegalaxies/155","moviegalaxies/157","moviegalaxies/158","moviegalaxies/159","moviegalaxies/161","moviegalaxies/162","moviegalaxies/163","moviegalaxies/164","moviegalaxies/165","moviegalaxies/166","moviegalaxies/167","moviegalaxies/169","moviegalaxies/170","moviegalaxies/171","moviegalaxies/173","moviegalaxies/174","moviegalaxies/175","moviegalaxies/176","moviegalaxies/177","moviegalaxies/178","moviegalaxies/179","moviegalaxies/180","moviegalaxies/182","moviegalaxies/183","moviegalaxies/184","moviegalaxies/185","moviegalaxies/186","moviegalaxies/187","moviegalaxies/189","moviegalaxies/190","moviegalaxies/191","moviegalaxies/192","moviegalaxies/193","moviegalaxies/194","moviegalaxies/195","moviegalaxies/197","moviegalaxies/198","moviegalaxies/199","moviegalaxies/200","moviegalaxies/201","moviegalaxies/204","moviegalaxies/205","moviegalaxies/206","moviegalaxies/208","moviegalaxies/209","moviegalaxies/210","moviegalaxies/211","moviegalaxies/212","moviegalaxies/213","moviegalaxies/214","moviegalaxies/216","moviegalaxies/217","moviegalaxies/218","moviegalaxies/220","moviegalaxies/221","moviegalaxies/222","moviegalaxies/223","moviegalaxies/224","moviegalaxies/225","moviegalaxies/226","moviegalaxies/227","moviegalaxies/229","moviegalaxies/231","moviegalaxies/232","moviegalaxies/234","moviegalaxies/235","moviegalaxies/236","moviegalaxies/238","moviegalaxies/239","moviegalaxies/240","moviegalaxies/241","moviegalaxies/242","moviegalaxies/243","moviegalaxies/244","moviegalaxies/246","moviegalaxies/247","moviegalaxies/248","moviegalaxies/249","moviegalaxies/250","moviegalaxies/251","moviegalaxies/252","moviegalaxies/253","moviegalaxies/254","moviegalaxies/255","moviegalaxies/256","moviegalaxies/257","moviegalaxies/258","moviegalaxies/260","moviegalaxies/261","moviegalaxies/262","moviegalaxies/263","moviegalaxies/264","moviegalaxies/265","moviegalaxies/266","moviegalaxies/268","moviegalaxies/269","moviegalaxies/271","moviegalaxies/272","moviegalaxies/273","moviegalaxies/274","moviegalaxies/276","moviegalaxies/278","moviegalaxies/279","moviegalaxies/280","moviegalaxies/281","moviegalaxies/282","moviegalaxies/283","moviegalaxies/286","moviegalaxies/287","moviegalaxies/288","moviegalaxies/289","moviegalaxies/290","moviegalaxies/291","moviegalaxies/292","moviegalaxies/293","moviegalaxies/294","moviegalaxies/295","moviegalaxies/296","moviegalaxies/297","moviegalaxies/298","moviegalaxies/299","moviegalaxies/300","moviegalaxies/301","moviegalaxies/302","moviegalaxies/303","moviegalaxies/304","moviegalaxies/305","moviegalaxies/306","moviegalaxies/307","moviegalaxies/308","moviegalaxies/309","moviegalaxies/310","moviegalaxies/311","moviegalaxies/313","moviegalaxies/314","moviegalaxies/315","moviegalaxies/316","moviegalaxies/317","moviegalaxies/318","moviegalaxies/319","moviegalaxies/320","moviegalaxies/321","moviegalaxies/322","moviegalaxies/323","moviegalaxies/324","moviegalaxies/325","moviegalaxies/326","moviegalaxies/327","moviegalaxies/328","moviegalaxies/329","moviegalaxies/330","moviegalaxies/331","moviegalaxies/332","moviegalaxies/333","moviegalaxies/334","moviegalaxies/335","moviegalaxies/336","moviegalaxies/337","moviegalaxies/338","moviegalaxies/339","moviegalaxies/340","moviegalaxies/341","moviegalaxies/343","moviegalaxies/344","moviegalaxies/346","moviegalaxies/347","moviegalaxies/348","moviegalaxies/349","moviegalaxies/350","moviegalaxies/351","moviegalaxies/352","moviegalaxies/354","moviegalaxies/355","moviegalaxies/356","moviegalaxies/357","moviegalaxies/358","moviegalaxies/359","moviegalaxies/361","moviegalaxies/362","moviegalaxies/363","moviegalaxies/364","moviegalaxies/366","moviegalaxies/368","moviegalaxies/369","moviegalaxies/370","moviegalaxies/371","moviegalaxies/372","moviegalaxies/373","moviegalaxies/374","moviegalaxies/375","moviegalaxies/376","moviegalaxies/377","moviegalaxies/378","moviegalaxies/379","moviegalaxies/383","moviegalaxies/384","moviegalaxies/385","moviegalaxies/386","moviegalaxies/387","moviegalaxies/388","moviegalaxies/389","moviegalaxies/391","moviegalaxies/393","moviegalaxies/394","moviegalaxies/395","moviegalaxies/396","moviegalaxies/397","moviegalaxies/398","moviegalaxies/399","moviegalaxies/400","moviegalaxies/401","moviegalaxies/402","moviegalaxies/403","moviegalaxies/404","moviegalaxies/405","moviegalaxies/406","moviegalaxies/407","moviegalaxies/408","moviegalaxies/409","moviegalaxies/410","moviegalaxies/411","moviegalaxies/412","moviegalaxies/413","moviegalaxies/414","moviegalaxies/415","moviegalaxies/416","moviegalaxies/417","moviegalaxies/418","moviegalaxies/419","moviegalaxies/420","moviegalaxies/421","moviegalaxies/422","moviegalaxies/423","moviegalaxies/424","moviegalaxies/426","moviegalaxies/427","moviegalaxies/429","moviegalaxies/430","moviegalaxies/432","moviegalaxies/433","moviegalaxies/434","moviegalaxies/435","moviegalaxies/436","moviegalaxies/437","moviegalaxies/438","moviegalaxies/439","moviegalaxies/442","moviegalaxies/443","moviegalaxies/444","moviegalaxies/445","moviegalaxies/446","moviegalaxies/447","moviegalaxies/449","moviegalaxies/450","moviegalaxies/451","moviegalaxies/452","moviegalaxies/453","moviegalaxies/454","moviegalaxies/455","moviegalaxies/456","moviegalaxies/457","moviegalaxies/458","moviegalaxies/459","moviegalaxies/460","moviegalaxies/461","moviegalaxies/462","moviegalaxies/464","moviegalaxies/465","moviegalaxies/466","moviegalaxies/467","moviegalaxies/468","moviegalaxies/469","moviegalaxies/470","moviegalaxies/471","moviegalaxies/472","moviegalaxies/474","moviegalaxies/475","moviegalaxies/476","moviegalaxies/477","moviegalaxies/478","moviegalaxies/481","moviegalaxies/482","moviegalaxies/483","moviegalaxies/484","moviegalaxies/485","moviegalaxies/486","moviegalaxies/487","moviegalaxies/488","moviegalaxies/489","moviegalaxies/490","moviegalaxies/491","moviegalaxies/494","moviegalaxies/495","moviegalaxies/496","moviegalaxies/497","moviegalaxies/498","moviegalaxies/499","moviegalaxies/500","moviegalaxies/501","moviegalaxies/503","moviegalaxies/505","moviegalaxies/506","moviegalaxies/507","moviegalaxies/508","moviegalaxies/509","moviegalaxies/510","moviegalaxies/511","moviegalaxies/512","moviegalaxies/513","moviegalaxies/515","moviegalaxies/516","moviegalaxies/517","moviegalaxies/518","moviegalaxies/519","moviegalaxies/520","moviegalaxies/521","moviegalaxies/523","moviegalaxies/524","moviegalaxies/525","moviegalaxies/526","moviegalaxies/527","moviegalaxies/528","moviegalaxies/529","moviegalaxies/530","moviegalaxies/531","moviegalaxies/532","moviegalaxies/533","moviegalaxies/534","moviegalaxies/535","moviegalaxies/536","moviegalaxies/537","moviegalaxies/538","moviegalaxies/539","moviegalaxies/541","moviegalaxies/542","moviegalaxies/543","moviegalaxies/544","moviegalaxies/545","moviegalaxies/547","moviegalaxies/548","moviegalaxies/549","moviegalaxies/550","moviegalaxies/551","moviegalaxies/552","moviegalaxies/553","moviegalaxies/555","moviegalaxies/556","moviegalaxies/558","moviegalaxies/559","moviegalaxies/560","moviegalaxies/562","moviegalaxies/564","moviegalaxies/565","moviegalaxies/566","moviegalaxies/567","moviegalaxies/569","moviegalaxies/570","moviegalaxies/571","moviegalaxies/572","moviegalaxies/573","moviegalaxies/575","moviegalaxies/577","moviegalaxies/578","moviegalaxies/579","moviegalaxies/580","moviegalaxies/581","moviegalaxies/582","moviegalaxies/583","moviegalaxies/584","moviegalaxies/586","moviegalaxies/587","moviegalaxies/591","moviegalaxies/592","moviegalaxies/593","moviegalaxies/594","moviegalaxies/595","moviegalaxies/599","moviegalaxies/600","moviegalaxies/601","moviegalaxies/602","moviegalaxies/603","moviegalaxies/605","moviegalaxies/606","moviegalaxies/607","moviegalaxies/608","moviegalaxies/609","moviegalaxies/610","moviegalaxies/611","moviegalaxies/615","moviegalaxies/616","moviegalaxies/618","moviegalaxies/619","moviegalaxies/620","moviegalaxies/621","moviegalaxies/622","moviegalaxies/623","moviegalaxies/624","moviegalaxies/625","moviegalaxies/628","moviegalaxies/629","moviegalaxies/630","moviegalaxies/631","moviegalaxies/632","moviegalaxies/633","moviegalaxies/634","moviegalaxies/635","moviegalaxies/636","moviegalaxies/637","moviegalaxies/638","moviegalaxies/640","moviegalaxies/641","moviegalaxies/642","moviegalaxies/643","moviegalaxies/644","moviegalaxies/645","moviegalaxies/647","moviegalaxies/648","moviegalaxies/649","moviegalaxies/650","moviegalaxies/651","moviegalaxies/652","moviegalaxies/653","moviegalaxies/655","moviegalaxies/656","moviegalaxies/657","moviegalaxies/658","moviegalaxies/659","moviegalaxies/660","moviegalaxies/661","moviegalaxies/662","moviegalaxies/664","moviegalaxies/665","moviegalaxies/666","moviegalaxies/667","moviegalaxies/668","moviegalaxies/669","moviegalaxies/670","moviegalaxies/671","moviegalaxies/672","moviegalaxies/673","moviegalaxies/675","moviegalaxies/677","moviegalaxies/678","moviegalaxies/679","moviegalaxies/682","moviegalaxies/683","moviegalaxies/685","moviegalaxies/686","moviegalaxies/687","moviegalaxies/688","moviegalaxies/689","moviegalaxies/691","moviegalaxies/692","moviegalaxies/693","moviegalaxies/694","moviegalaxies/695","moviegalaxies/696","moviegalaxies/697","moviegalaxies/698","moviegalaxies/699","moviegalaxies/700","moviegalaxies/701","moviegalaxies/703","moviegalaxies/704","moviegalaxies/705","moviegalaxies/706","moviegalaxies/707","moviegalaxies/708","moviegalaxies/710","moviegalaxies/712","moviegalaxies/713","moviegalaxies/714","moviegalaxies/715","moviegalaxies/716","moviegalaxies/717","moviegalaxies/718","moviegalaxies/719","moviegalaxies/720","moviegalaxies/721","moviegalaxies/723","moviegalaxies/724","moviegalaxies/725","moviegalaxies/727","moviegalaxies/728","moviegalaxies/729","moviegalaxies/730","moviegalaxies/731","moviegalaxies/732","moviegalaxies/733","moviegalaxies/734","moviegalaxies/735","moviegalaxies/737","moviegalaxies/738","moviegalaxies/739","moviegalaxies/740","moviegalaxies/741","moviegalaxies/742","moviegalaxies/743","moviegalaxies/744","moviegalaxies/745","moviegalaxies/746","moviegalaxies/747","moviegalaxies/748","moviegalaxies/749","moviegalaxies/750","moviegalaxies/751","moviegalaxies/753","moviegalaxies/754","moviegalaxies/755","moviegalaxies/756","moviegalaxies/757","moviegalaxies/758","moviegalaxies/760","moviegalaxies/761","moviegalaxies/762","moviegalaxies/764","moviegalaxies/765","moviegalaxies/766","moviegalaxies/767","moviegalaxies/768","moviegalaxies/769","moviegalaxies/770","moviegalaxies/771","moviegalaxies/772","moviegalaxies/773","moviegalaxies/774","moviegalaxies/775","moviegalaxies/777","moviegalaxies/779","moviegalaxies/780","moviegalaxies/781","moviegalaxies/782","moviegalaxies/783","moviegalaxies/784","moviegalaxies/785","moviegalaxies/786","moviegalaxies/787","moviegalaxies/788","moviegalaxies/790","moviegalaxies/791","moviegalaxies/792","moviegalaxies/793","moviegalaxies/794","moviegalaxies/797","moviegalaxies/798","moviegalaxies/799","moviegalaxies/800","moviegalaxies/801","moviegalaxies/802","moviegalaxies/803","moviegalaxies/804","moviegalaxies/805","moviegalaxies/806","moviegalaxies/808","moviegalaxies/809","moviegalaxies/810","moviegalaxies/811","moviegalaxies/812","moviegalaxies/814","moviegalaxies/815","moviegalaxies/816","moviegalaxies/817","moviegalaxies/818","moviegalaxies/819","moviegalaxies/820","moviegalaxies/821","moviegalaxies/822","moviegalaxies/824","moviegalaxies/825","moviegalaxies/826","moviegalaxies/827","moviegalaxies/828","moviegalaxies/829","moviegalaxies/830","moviegalaxies/831","moviegalaxies/832","moviegalaxies/833","moviegalaxies/834","moviegalaxies/835","moviegalaxies/836","moviegalaxies/837","moviegalaxies/839","moviegalaxies/840","moviegalaxies/841","moviegalaxies/843","moviegalaxies/844","moviegalaxies/846","moviegalaxies/847","moviegalaxies/848","moviegalaxies/849","moviegalaxies/850","moviegalaxies/851","moviegalaxies/852","moviegalaxies/853","moviegalaxies/854","moviegalaxies/855","moviegalaxies/857","moviegalaxies/858","moviegalaxies/859","moviegalaxies/860","moviegalaxies/861","moviegalaxies/862","moviegalaxies/863","moviegalaxies/865","moviegalaxies/867","moviegalaxies/869","moviegalaxies/870","moviegalaxies/871","moviegalaxies/872","moviegalaxies/873","moviegalaxies/874","moviegalaxies/875","moviegalaxies/876","moviegalaxies/877","moviegalaxies/878","moviegalaxies/879","moviegalaxies/880","moviegalaxies/881","moviegalaxies/883","moviegalaxies/884","moviegalaxies/885","moviegalaxies/887","moviegalaxies/888","moviegalaxies/889","moviegalaxies/891","moviegalaxies/892","moviegalaxies/893","moviegalaxies/895","moviegalaxies/896","moviegalaxies/899","moviegalaxies/901","moviegalaxies/902","moviegalaxies/904","moviegalaxies/905","moviegalaxies/906","moviegalaxies/907","moviegalaxies/908","moviegalaxies/910","moviegalaxies/911","moviegalaxies/912","moviegalaxies/913","moviegalaxies/914",
#              "foodweb_baywet",
#              "plant_pol_vazquez/Llao Llao","plant_pol_vazquez/Cerro Lopez","plant_pol_vazquez/Safariland","plant_pol_vazquez/Arroyo Goye","plant_pol_vazquez/Mascardi (nc)","plant_pol_vazquez/Mascardi (c)","plant_pol_vazquez/Quetrihue (nc)","plant_pol_vazquez/Quetrihue (c)","plant_pol_vazquez/All sites pooled",
#              "dom/Adcock_2015a","dom/Adcock_2015b","dom/Alados_1992a","dom/Alados_1992b","dom/Alados_1992c","dom/Alados_1992d","dom/Alados_1992e","dom/Allee_1954","dom/Anderson_2016a","dom/Anderson_2016d","dom/Appleby_1983","dom/Archie_2006a","dom/Archie_2006b","dom/Archie_2006c","dom/Archie_2006d","dom/Archie_2006e","dom/Archie_2006f","dom/Archie_2006g","dom/Archie_2006h","dom/Archie_2006i","dom/Arlet_2015f","dom/Barette_1986","dom/Bennett_1939","dom/BergstromFedigan_2010b","dom/BergstromFedigan_2010c","dom/Berman_2004a","dom/Berman_2004b","dom/Berman_2004c","dom/Blatrix_2004b","dom/Blatrix_2004c","dom/Blatrix_2004d","dom/Blatrix_2004e","dom/Bonanni_2007a","dom/Bonanni_2007b","dom/Bonanni_2007c","dom/Bonanni_2017a","dom/Bonanni_2017b","dom/Bonanni_2017c","dom/Bonanni_2017d","dom/Bromley_1991","dom/Cafazzo_2010a","dom/Cafazzo_2010b","dom/Cheney_1977","dom/Cheney_1977b","dom/CluttonBrock_1976","dom/Collias_1950a","dom/Collias_1950b","dom/Collias_1950c","dom/Collias_1951a","dom/Collias_1951b","dom/Collias_1951c","dom/Correa_2013a","dom/Correa_2013d","dom/Cote_2000","dom/Cote_2000b","dom/Cote_2000c","dom/Cote_2000d","dom/Cui_2014","dom/David_2003a","dom/David_2003b","dom/DeLaFuente_2019","dom/DeLaO_2019a","dom/DeLaO_2019b","dom/DeLaO_2019c","dom/DeLaO_2019d","dom/deVries_2006","dom/deWaal_1977a","dom/deWaal_1977b","dom/deWaal_1985","dom/Diniz_2019","dom/Douglis_1948a","dom/Douglis_1948b","dom/Dubosq2013a","dom/Dubosq2013b","dom/Dubosq2013c","dom/Dubosq2013d","dom/Dubosq2013e","dom/Dubosq2013f","dom/Ellard_1989","dom/Essler_2016a","dom/Essler_2016c","dom/Essler_2016d","dom/Fairbanks_1994a","dom/Fairbanks_1994b","dom/Fairbanks_1994c","dom/Fairbanks_1994d","dom/Fairbanks_1994e","dom/Farentinos_1972a","dom/Farentinos_1972b","dom/Farentinos_1972c","dom/Farentinos_1972d","dom/Farentinos_1972e","dom/Farentinos_1972f","dom/Foerster_2016a","dom/Foerster_2016b","dom/Fournier_1995","dom/Fournier_1995b","dom/Frank_1986a","dom/Frank_1986b","dom/Franz_2015a","dom/Franz_2015b","dom/Franz_2015c","dom/Franz_2015d","dom/Franz_2015e","dom/Funkhouser_2018a","dom/Funkhouser_2018b","dom/Harcourt_1979","dom/Harcourt_1989","dom/Hartzler_1970","dom/Hass_1991","dom/Hass_1991b","dom/Hass_1991c","dom/Hausfater_1975a","dom/Hausfater_1975b","dom/Hausfater_1975c","dom/Hausfater_1982a","dom/Hausfater_1982b","dom/Hayaki_1989a","dom/Hayaki_1989b","dom/Heitor_2006","dom/HeitorVicente_2010","dom/Hewitt_2009a","dom/Hewitt_2009b","dom/Hewitt_2009c","dom/Hewitt_2009d","dom/Hirotani_1994","dom/Hirsch_2007a","dom/Hirsch_2007b","dom/Hirsch_2007c","dom/Hirsch_2007d","dom/Hobson_2015a","dom/Hobson_2015b","dom/Holekamp_1991","dom/Holekamp_1993a","dom/Holekamp_1993b","dom/Isbell_1998a","dom/Isbell_1998b","dom/Ito_1993a","dom/Ito_1993b","dom/Ito_1993c","dom/Izar_2006","dom/IzawaWatanabe_2011","dom/Jenks_1995a","dom/Jenks_1995b","dom/Johnson_2017","dom/Jones_1980a","dom/Jones_1980b","dom/Kaufmann_1974","dom/Kikkawa_1980","dom/Koenig_2004","dom/Kohda_1991","dom/Kolodziejczyk_2005","dom/Korstjens_2002","dom/Koutnik_1981","dom/Koutnik_1981b","dom/Kurvers_2009","dom/Lahti_1994a","dom/Lahti_1994b","dom/Langbein_2004a","dom/Langbein_2004b","dom/Langbein_2004c","dom/Langbein_2004d","dom/Langley_2018","dom/Lee_1979a","dom/Lee_1979b","dom/Lee_2017a","dom/Lee_2017b","dom/Lee_2017c","dom/Lee_2017d","dom/Lee_2017e","dom/Lee_2017f","dom/Lee_2018a","dom/Lee_2018b","dom/Lee_2018c","dom/Lee_2018d","dom/Lee_2018e","dom/Lee_2019a","dom/Lee_2019b","dom/Lee_2019c","dom/Lee_2019d","dom/Lee_2019e","dom/Lott_1979","dom/Lott_1987","dom/Lu_2008a","dom/Lu_2008b","dom/Malherbe_2007","dom/Marler_1955","dom/Masure_1934","dom/Masure_1934a","dom/Masure_1934b","dom/Mather_1985","dom/Matsuda_2012","dom/McCune_2019a","dom/McCune_2019c","dom/McCune_2019d","dom/McCune_2019e","dom/McCune_2019f","dom/McDougall_2010","dom/McMahan_1984","dom/Miller_2010a","dom/Miller_2010b","dom/Miller_2010c","dom/Moller_1987a","dom/Moller_1987b","dom/Moller_1987c","dom/Monnin_1999","dom/Moore_1978a","dom/Moore_1978b","dom/Murray_2007","dom/Mwamende_2009a","dom/Mwamende_2009b","dom/Myrberg_1972","dom/Myrberg_1974","dom/Nakano_1994","dom/Nakano_1995","dom/Natoli_1991","dom/Nelissen_1985a","dom/Nelissen_1985b","dom/NewtonFisher_2004a","dom/NewtonFisher_2004b","dom/Norscia_2015a","dom/Norscia_2015b","dom/Norscia_2015c","dom/Norscia_2015d","dom/Norscia_2015e","dom/Norscia_2015f","dom/Norscia_2015g","dom/Norscia_2015h","dom/Norscia_2015i","dom/Norscia_2015j","dom/Ortius_1995","dom/Oshea_1976","dom/Owens_1996","dom/Painter_2018","dom/Paoli_2006","dom/Pardi_1948a","dom/Pardi_1948b","dom/Parsons_1980a","dom/Parsons_1980b","dom/Patterson_1977","dom/Payne_2003","dom/Poisbleau_2005a","dom/Poisbleau_2005b","dom/Poisbleau_2005c","dom/Poisbleau_2006a","dom/Poisbleau_2006b","dom/Post_1992","dom/Prieto_1978","dom/Reason_1988","dom/Richter_2009","dom/RizaldiWatanabe_2010a","dom/RizaldiWatanabe_2010b","dom/Robbins_2008","dom/Roell_1978","dom/Rossler_2017","dom/Rovero_1999a","dom/Rovero_1999b","dom/Russell_1970a","dom/Russell_1970b","dom/Rutberg_1986a","dom/Rutberg_1986b","dom/Samuels_1987","dom/Samuels_1987b","dom/Sandel_2017a","dom/Sandel_2017b","dom/SatohOhkawara_2008a","dom/SatohOhkawara_2008b","dom/SatohOhkawara_2008c","dom/ScheinFohrman_1955","dom/SchjelderupEbbe_1922a","dom/SchjelderupEbbe_1922b","dom/ScottLockhard_1999a","dom/ScottLockhard_1999b","dom/Seibert_2001","dom/Setchell_2005a","dom/Setchell_2005b","dom/SetiaVanSchaik_2007","dom/Seyfarth_1976a","dom/Seyfarth_1976b","dom/Seyfarth_1976c","dom/Seyfarth_1976d","dom/Seyfarth_1976e","dom/Seyfarth_1976f","dom/Sharpe_2013a","dom/Sharpe_2013b","dom/Sharpe_2013c","dom/Sharpe_2013d","dom/Shimoji_2014a","dom/Shimoji_2014b","dom/Shimoji_2014c","dom/Shoemaker_1939","dom/Silk_2019a","dom/Slotow_1993","dom/Smith_1976","dom/Smith_2011a","dom/Solberg_1997a","dom/Solberg_1997b","dom/Solberg_1997c","dom/SomersNel_1998a","dom/SomersNel_1998b","dom/SomersNel_1998c","dom/Stamps_1978","dom/Sterck_1997","dom/Strauss_2019a","dom/Strauss_2019b","dom/Strauss_2019c","dom/Strauss_2019d","dom/StrayerCummins_1980","dom/StrayerCummins_1980b","dom/StrayerCummins_1980c","dom/StrayerCummins_1980d","dom/Struhsaker_1967a","dom/Struhsaker_1967b","dom/Sullivan_1982","dom/Surbeck_2011","dom/SurbeckHohmann_2013b","dom/Tamm_1977","dom/Tamura_1988","dom/Tarvin_1997","dom/Thompson_1960a","dom/Thompson_1960b","dom/Thompson_1960c","dom/Thompson_1960d","dom/Thompson_1960e","dom/Thompson_1993a","dom/Thompson_1993b","dom/Tilson_1984","dom/Tong_2020a","dom/Tong_2020b","dom/Tong_2020c","dom/Tong_2020d","dom/Tong_2020e","dom/Tong_2020f","dom/Tong_2020g","dom/Tong_2020h","dom/Tong_2020i","dom/Tong_2020j","dom/Torr_1996a","dom/Torr_1996b","dom/Torr_1996c","dom/Trebouet_2019a","dom/Trebouet_2019b","dom/Trebouet_2019c","dom/Trebouet_2019d","dom/Trunzer_1999","dom/Utt_2008a","dom/ValLaillet_2008a","dom/ValLaillet_2008b","dom/ValLaillet_2008c","dom/ValLaillet_2008d","dom/ValLaillet_2008e","dom/ValLaillet_2008f","dom/VanDessel_2018a","dom/VanDessel_2018b","dom/VanDierendonck_1995","dom/VanHooffWensing_1987","dom/Varley_1966","dom/Varley_1966b","dom/Vervaecke_2000","dom/Vervaecke_2010a","dom/Vervaecke_2010b","dom/Vervaecke_2010c","dom/Vervaecke_2010d","dom/Vilette_2020","dom/Waterhouse_1976a","dom/Waterhouse_1976b","dom/Waterhouse_1976c","dom/Watson_1970a","dom/Watson_1970b","dom/Watt_1986a","dom/Watt_1986b","dom/Watt_1986c","dom/Watt_1986d","dom/Watt_1986e","dom/Watt_1986f","dom/Watts_1985","dom/Watts_1994a","dom/Watts_1994b","dom/Watts_1994c","dom/Wells_1979","dom/WestEberhard_1986","dom/White_2007a","dom/White_2007b","dom/White_2007c","dom/Wikberg_2013a","dom/Wikberg_2013b","dom/Wikberg_2013c","dom/Wikberg_2013d","dom/Wikberg_2013e","dom/Wikberg_2013f","dom/Williams_1972","dom/Williamson_2016a","dom/Williamson_2016b","dom/Williamson_2016c","dom/Williamson_2016d","dom/Williamson_2016e","dom/Williamson_2016f","dom/Williamson_2016g","dom/Williamson_2016h","dom/Williamson_2016i","dom/Williamson_2016j","dom/Williamson_2016k","dom/Williamson_2017a","dom/Williamson_2017b","dom/Williamson_2017c","dom/Williamson_2017d","dom/Williamson_2017e","dom/Williamson_2017f","dom/Williamson_2017g","dom/Williamson_2017h","dom/Williamson_2017i","dom/Williamson_2017j","dom/Williamson_2017k","dom/Williamson_2017l","dom/Williamson_2017m","dom/Williamson_2017n","dom/Williamson_2017o","dom/Williamson_2019a","dom/Williamson_2019b","dom/Williamson_2019c","dom/Williamson_2019d","dom/Williamson_2019e","dom/Williamson_2019f","dom/Williamson_2019g","dom/Williamson_2019h","dom/Wittemeyer_2007","dom/Wittig_2003","dom/Yasukawa_1983a","dom/Yasukawa_1983b","dom/Yasukawa_1983c","dom/Yasukawa_1983d","dom/Yasukawa_1983e","dom/Yasukawa_1983f","dom/Yasukawa_1983g","dom/Yasukawa_1983h","dom/Zine_2000",
#              "cintestinalis",
#              "faculty_hiring/computer_science","faculty_hiring/business","faculty_hiring/history",
#              "fao_trade","residence_hall","un_migrations","sp_primary_school","celegansneural",
#              "sp_high_school/proximity","sp_high_school/diaries","sp_high_school/survey","sp_high_school/facebook",
#              "london_transport",
#              "sp_colocation/Thiers13","sp_colocation/SFHH","sp_colocation/LyonSchool","sp_colocation/LH10","sp_colocation/InVS15","sp_colocation/InVS13",
#              "webkb/webkb_wisconsin_link1","webkb/webkb_wisconsin_cocite","webkb/webkb_washington_link1","webkb/webkb_washington_cocite","webkb/webkb_cornell_link1","webkb/webkb_cornell_cocite","webkb/webkb_texas_link1","webkb/webkb_texas_cocite",
#             "celegans_2019/male_chemical","celegans_2019/male_gap_junction","celegans_2019/hermaphrodite_chemical","celegans_2019/hermaphrodite_gap_junction","celegans_2019/male_chemical_synapse","celegans_2019/male_gap_junction_synapse","celegans_2019/hermaphrodite_chemical_synapse","celegans_2019/hermaphrodite_gap_junction_synapse","celegans_2019/male_chemical_corrected","celegans_2019/male_gap_junction_corrected","celegans_2019/hermaphrodite_chemical_corrected","celegans_2019/hermaphrodite_gap_junction_corrected",
#             #"messal_shale", #DOES NOT WORK
#              "plant_pol_kato",
#              "copenhagen/sms","copenhagen/fb_friends","copenhagen/calls","copenhagen/bt",
#             "product_space/HS","product_space/SITC",
#             "unicodelang",
#             "budapest_connectome/all_20k","budapest_connectome/all_200k","budapest_connectome/all_1m","budapest_connectome/female_20k","budapest_connectome/female_200k","budapest_connectome/female_1m","budapest_connectome/male_20k","budapest_connectome/male_200k","budapest_connectome/male_1m",
#             "new_zealand_collab","netscience","bible_nouns",
#             # "add_health/comm1","add_health/comm2","add_health/comm3","add_health/comm4",#there are about 100 graphs. we consider only 4. BUT IT DOES NOT WORK: files offline
#             "openflights","bitcoin_alpha","bitcoin_trust","advogato","foldoc",
#             "physics_collab/pierreAuger","physics_collab/arXiv",
#             "escorts","fly_hemibrain",
#             "arxiv_collab/cond-mat-1999","arxiv_collab/cond-mat-2003","arxiv_collab/cond-mat-2005","arxiv_collab/astro-ph-1999","arxiv_collab/hep-th-1999",
#             "us_agencies/alabama","us_agencies/alaska","us_agencies/arizona","us_agencies/arkansas","us_agencies/california","us_agencies/colorado","us_agencies/connecticut","us_agencies/delaware","us_agencies/florida","us_agencies/georgia","us_agencies/hawaii","us_agencies/idaho","us_agencies/illinois","us_agencies/indiana","us_agencies/iowa","us_agencies/kansas","us_agencies/kentucky","us_agencies/louisiana","us_agencies/maine","us_agencies/maryland","us_agencies/massachusetts","us_agencies/michigan","us_agencies/minnesota","us_agencies/mississippi","us_agencies/missouri","us_agencies/montana","us_agencies/nebraska","us_agencies/nevada","us_agencies/newhampshire","us_agencies/newjersey","us_agencies/newmexico","us_agencies/newyork","us_agencies/northcarolina","us_agencies/northdakota","us_agencies/ohio","us_agencies/oklahoma","us_agencies/oregon","us_agencies/pennsylvania","us_agencies/rhodeisland","us_agencies/southcarolina","us_agencies/southdakota","us_agencies/tennessee","us_agencies/texas","us_agencies/utah","us_agencies/vermont","us_agencies/virginia","us_agencies/washington","us_agencies/westvirginia","us_agencies/wisconsin","us_agencies/wyoming","us_agencies/aggregate",
#            "eu_procurements_alt/AT_2008","eu_procurements_alt/AT_2009","eu_procurements_alt/AT_2010","eu_procurements_alt/AT_2011","eu_procurements_alt/AT_2012","eu_procurements_alt/AT_2013","eu_procurements_alt/AT_2014","eu_procurements_alt/AT_2015","eu_procurements_alt/AT_2016","eu_procurements_alt/BE_2008","eu_procurements_alt/BE_2009","eu_procurements_alt/BE_2010","eu_procurements_alt/BE_2011","eu_procurements_alt/BE_2012","eu_procurements_alt/BE_2013","eu_procurements_alt/BE_2014","eu_procurements_alt/BE_2015","eu_procurements_alt/BE_2016","eu_procurements_alt/BG_2008","eu_procurements_alt/BG_2009","eu_procurements_alt/BG_2010","eu_procurements_alt/BG_2011","eu_procurements_alt/BG_2012","eu_procurements_alt/BG_2013","eu_procurements_alt/BG_2014","eu_procurements_alt/BG_2015","eu_procurements_alt/BG_2016","eu_procurements_alt/CY_2008","eu_procurements_alt/CY_2009","eu_procurements_alt/CY_2010","eu_procurements_alt/CY_2011","eu_procurements_alt/CY_2012","eu_procurements_alt/CY_2013","eu_procurements_alt/CY_2014","eu_procurements_alt/CY_2015","eu_procurements_alt/CY_2016","eu_procurements_alt/CZ_2008","eu_procurements_alt/CZ_2009","eu_procurements_alt/CZ_2010","eu_procurements_alt/CZ_2011","eu_procurements_alt/CZ_2012","eu_procurements_alt/CZ_2013","eu_procurements_alt/CZ_2014","eu_procurements_alt/CZ_2015","eu_procurements_alt/CZ_2016","eu_procurements_alt/DE_2008","eu_procurements_alt/DE_2009","eu_procurements_alt/DE_2010","eu_procurements_alt/DE_2011","eu_procurements_alt/DE_2012","eu_procurements_alt/DE_2013","eu_procurements_alt/DE_2014","eu_procurements_alt/DE_2015","eu_procurements_alt/DE_2016","eu_procurements_alt/DK_2008","eu_procurements_alt/DK_2009","eu_procurements_alt/DK_2010","eu_procurements_alt/DK_2011","eu_procurements_alt/DK_2012","eu_procurements_alt/DK_2013","eu_procurements_alt/DK_2014","eu_procurements_alt/DK_2015","eu_procurements_alt/DK_2016","eu_procurements_alt/EE_2008","eu_procurements_alt/EE_2009","eu_procurements_alt/EE_2010","eu_procurements_alt/EE_2011","eu_procurements_alt/EE_2012","eu_procurements_alt/EE_2013","eu_procurements_alt/EE_2014","eu_procurements_alt/EE_2015","eu_procurements_alt/EE_2016","eu_procurements_alt/ES_2008","eu_procurements_alt/ES_2009","eu_procurements_alt/ES_2010","eu_procurements_alt/ES_2011","eu_procurements_alt/ES_2012","eu_procurements_alt/ES_2013","eu_procurements_alt/ES_2014","eu_procurements_alt/ES_2015","eu_procurements_alt/ES_2016","eu_procurements_alt/FI_2008","eu_procurements_alt/FI_2009","eu_procurements_alt/FI_2010","eu_procurements_alt/FI_2011","eu_procurements_alt/FI_2012","eu_procurements_alt/FI_2013","eu_procurements_alt/FI_2014","eu_procurements_alt/FI_2015","eu_procurements_alt/FI_2016","eu_procurements_alt/FR_2008","eu_procurements_alt/FR_2009","eu_procurements_alt/FR_2010","eu_procurements_alt/FR_2011","eu_procurements_alt/FR_2012","eu_procurements_alt/FR_2013","eu_procurements_alt/FR_2014","eu_procurements_alt/FR_2015","eu_procurements_alt/FR_2016","eu_procurements_alt/GR_2008","eu_procurements_alt/GR_2009","eu_procurements_alt/GR_2010","eu_procurements_alt/GR_2011","eu_procurements_alt/GR_2012","eu_procurements_alt/GR_2013","eu_procurements_alt/GR_2014","eu_procurements_alt/GR_2015","eu_procurements_alt/GR_2016","eu_procurements_alt/HU_2008","eu_procurements_alt/HU_2009","eu_procurements_alt/HU_2010","eu_procurements_alt/HU_2011","eu_procurements_alt/HU_2012","eu_procurements_alt/HU_2013","eu_procurements_alt/HU_2014","eu_procurements_alt/HU_2015","eu_procurements_alt/HU_2016","eu_procurements_alt/IE_2008","eu_procurements_alt/IE_2009","eu_procurements_alt/IE_2010","eu_procurements_alt/IE_2011","eu_procurements_alt/IE_2012","eu_procurements_alt/IE_2013","eu_procurements_alt/IE_2014","eu_procurements_alt/IE_2015","eu_procurements_alt/IE_2016","eu_procurements_alt/IT_2008","eu_procurements_alt/IT_2009","eu_procurements_alt/IT_2010","eu_procurements_alt/IT_2011","eu_procurements_alt/IT_2012","eu_procurements_alt/IT_2013","eu_procurements_alt/IT_2014","eu_procurements_alt/IT_2015","eu_procurements_alt/IT_2016","eu_procurements_alt/LT_2008","eu_procurements_alt/LT_2009","eu_procurements_alt/LT_2010","eu_procurements_alt/LT_2011","eu_procurements_alt/LT_2012","eu_procurements_alt/LT_2013","eu_procurements_alt/LT_2014","eu_procurements_alt/LT_2015","eu_procurements_alt/LT_2016","eu_procurements_alt/LV_2008","eu_procurements_alt/LV_2009","eu_procurements_alt/LV_2010","eu_procurements_alt/LV_2011","eu_procurements_alt/LV_2012","eu_procurements_alt/LV_2013","eu_procurements_alt/LV_2014","eu_procurements_alt/LV_2015","eu_procurements_alt/LV_2016","eu_procurements_alt/NL_2008","eu_procurements_alt/NL_2009","eu_procurements_alt/NL_2010","eu_procurements_alt/NL_2011","eu_procurements_alt/NL_2012","eu_procurements_alt/NL_2013","eu_procurements_alt/NL_2014","eu_procurements_alt/NL_2015","eu_procurements_alt/NL_2016","eu_procurements_alt/NO_2008","eu_procurements_alt/NO_2009","eu_procurements_alt/NO_2010","eu_procurements_alt/NO_2011","eu_procurements_alt/NO_2012","eu_procurements_alt/NO_2013","eu_procurements_alt/NO_2014","eu_procurements_alt/NO_2015","eu_procurements_alt/NO_2016","eu_procurements_alt/PL_2008","eu_procurements_alt/PL_2009","eu_procurements_alt/PL_2010","eu_procurements_alt/PL_2011","eu_procurements_alt/PL_2012","eu_procurements_alt/PL_2013","eu_procurements_alt/PL_2014","eu_procurements_alt/PL_2015","eu_procurements_alt/PL_2016","eu_procurements_alt/PT_2008","eu_procurements_alt/PT_2009","eu_procurements_alt/PT_2010","eu_procurements_alt/PT_2011","eu_procurements_alt/PT_2012","eu_procurements_alt/PT_2013","eu_procurements_alt/PT_2014","eu_procurements_alt/PT_2015","eu_procurements_alt/PT_2016","eu_procurements_alt/RO_2008","eu_procurements_alt/RO_2009","eu_procurements_alt/RO_2010","eu_procurements_alt/RO_2011","eu_procurements_alt/RO_2012","eu_procurements_alt/RO_2013","eu_procurements_alt/RO_2014","eu_procurements_alt/RO_2015","eu_procurements_alt/RO_2016","eu_procurements_alt/SE_2008","eu_procurements_alt/SE_2009","eu_procurements_alt/SE_2010","eu_procurements_alt/SE_2011","eu_procurements_alt/SE_2012","eu_procurements_alt/SE_2013","eu_procurements_alt/SE_2014","eu_procurements_alt/SE_2015","eu_procurements_alt/SE_2016","eu_procurements_alt/SI_2008","eu_procurements_alt/SI_2009","eu_procurements_alt/SI_2010","eu_procurements_alt/SI_2011","eu_procurements_alt/SI_2012","eu_procurements_alt/SI_2013","eu_procurements_alt/SI_2014","eu_procurements_alt/SI_2015","eu_procurements_alt/SI_2016","eu_procurements_alt/SK_2008","eu_procurements_alt/SK_2009","eu_procurements_alt/SK_2010","eu_procurements_alt/SK_2011","eu_procurements_alt/SK_2012","eu_procurements_alt/SK_2013","eu_procurements_alt/SK_2014","eu_procurements_alt/SK_2015","eu_procurements_alt/SK_2016","eu_procurements_alt/UK_2008","eu_procurements_alt/UK_2009","eu_procurements_alt/UK_2010","eu_procurements_alt/UK_2011","eu_procurements_alt/UK_2012","eu_procurements_alt/UK_2013","eu_procurements_alt/UK_2014","eu_procurements_alt/UK_2015","eu_procurements_alt/UK_2016"
#             "jester","libimseti","wiki_article_words","bookcrossing",
#             "twitter_higgs/social","twitter_higgs/retweet","twitter_higgs/reply","twitter_higgs/mention","twitter_higgs/activity",
#             "eu_procurements"  ,"epinions",
#             #"yahoo_song", #DOES NOT WORK ON COLAB: TOO MUCH RAM
#              "mag_history_coauthor/max-25-bip","mag_history_coauthor/max-25-proj","mag_history_coauthor/full-bip","mag_history_coauthor/full-proj",
#              "mag_geology_coauthor/max-25-bip","mag_geology_coauthor/max-25-proj","mag_geology_coauthor/full-bip","mag_geology_coauthor/full-proj",
#              "amazon_ratings",
#              #"dblp_simplices/max-25-bip","dblp_simplices/max-25-proj","dblp_simplices/full-bip","dblp_simplices/full-proj",
#              #"bag_of_words/enron","bag_of_words/kos","bag_of_words/nips","bag_of_words/nytimes", #"bag_of_words/pubmed", TOO BIG
#              #"microsoft_concept"
# ]

#Unfortunately, not all models classified as weighted have weights. We handle these cases setting deafult weight 1
#model_names=["hens"] # DOES NOT HAVE WEIGHTS! The default '1' will be set to all edges

prefix_to_weight={'eu_procurements_alt/':'pctSingleBid','arxiv_collab/':'value','bag_of_words/':'count',
                  'budapest_connectome/':'occurences','celegans_2019':'connectivity',
                  'us_agencies/':'link_counts','plant_pol_vazquez':'count'}

#Set flags on where to store the obtained CSV files
download_file=True
store_on_gdrive=True
path_g_drive="SIS_OnWeighted/"
export_SISOnweightedGraphs_erode(model_names,download_file,store_on_gdrive,path_g_drive,prefix_to_weight)

# If interested in drawing

In [ ]:
# from graph_tool.all import *
# g = collection.data["celegansneural"]
# state = minimize_nested_blockmodel_dl(g)
# state = minimize_nested_blockmodel_dl(g)
# state.draw()